In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Unzip the archive
local_zip = '/content/drive/MyDrive/Leaf Data/LEAF_BETA.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()

In [ ]:
import os

base_dir = '/content/LEAF_BETA'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/training_set'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/validation_set'))

Contents of base directory:
['validation_set', 'training_set']

Contents of train directory:
['Healthy Leaves', 'Stage1', 'Stage4', 'Stage2', 'Stage3']

Contents of validation directory:
['Healthy Leaves', 'Stage1', 'Stage4', 'Stage2', 'Stage3']


In [ ]:
import os

train_dir = os.path.join(base_dir, 'training_set')
validation_dir = os.path.join(base_dir, 'validation_set')


train_h_dir = os.path.join(train_dir, 'Healthy Leaves')
train_one_dir = os.path.join(train_dir, 'Stage1')
train_two_dir = os.path.join(train_dir, 'Stage2')
train_three_dir = os.path.join(train_dir, 'Stage3')
train_four_dir = os.path.join(train_dir, 'Stage4')

validation_h_dir = os.path.join(validation_dir, 'Healthy Leaves')
validation_one_dir = os.path.join(validation_dir, 'Stage1')
validation_two_dir = os.path.join(validation_dir, 'Stage2')
validation_three_dir = os.path.join(validation_dir, 'Stage3')
validation_four_dir = os.path.join(validation_dir, 'Stage4')

In [ ]:
train_h_fnames = os.listdir( train_h_dir )
train_one_fnames = os.listdir( train_one_dir )
train_two_fnames = os.listdir( train_two_dir )
train_three_fnames = os.listdir( train_three_dir )
train_four_fnames = os.listdir( train_four_dir )

print(train_h_fnames[:10])
print(train_one_fnames[:10])
print(train_two_fnames[:10])
print(train_three_fnames[:10])
print(train_four_fnames[:10])

['WhatsApp Image 2022-07-04 at 21.03.0617532.jpeg', 'WhatsApp Image 2022-07-04 at 21.03.0617590.jpeg', 'WhatsApp Image 2022-07-04 at 21.03.0618598.jpeg', 'WhatsApp Image 2022-07-04 at 21.03.0618530.jpeg', 'WhatsApp Image 2022-07-04 at 21.03.0617022.jpeg', 'WhatsApp Image 2022-07-04 at 21.03.0617515.jpeg', 'WhatsApp Image 2022-07-04 at 21.03.0618036.jpeg', 'WhatsApp Image 2022-07-04 at 21.03.0618028.jpeg', 'WhatsApp Image 2022-07-04 at 21.03.0617072.jpeg', 'WhatsApp Image 2022-07-04 at 21.03.0618518.jpeg']
['WhatsApp Image 2022-07-04 at 21.21.57195131.jpeg', 'WhatsApp Image 2022-07-04 at 21.21.57190295.jpeg', 'WhatsApp Image 2022-07-04 at 21.21.57190197.jpeg', 'WhatsApp Image 2022-07-04 at 21.21.57185330.jpeg', 'WhatsApp Image 2022-07-04 at 21.21.57185320.jpeg', 'WhatsApp Image 2022-07-04 at 21.21.57185322.jpeg', 'WhatsApp Image 2022-07-04 at 21.21.57190206.jpeg', 'WhatsApp Image 2022-07-04 at 21.21.57190124.jpeg', 'WhatsApp Image 2022-07-04 at 21.21.57190316.jpeg', 'WhatsApp Image 2022

In [ ]:
import tensorflow as tf
from keras.applications import MobileNetV3Small
model = tf.keras.applications.mobilenet.MobileNet(
    input_shape=(224,224,3),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=5,
    classifier_activation='softmax'
)

In [ ]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [ ]:
for i in model.layers[:-1]:
  i.trainable= True

In [ ]:
x = model.layers[-1].output
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Conv2D(2,3)(x)
x = tf.keras.layers.Conv2D(2,3)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.1)(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
pred = tf.keras.layers.Dense(5, activation='softmax')(x)

In [ ]:
from tensorflow.keras import Model
m = Model(model.input, pred)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    class_mode='categorical',
                                                    target_size=(224, 224))
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         class_mode  = 'categorical',
                                                         target_size = (224, 224))

Found 2000 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
import os
from PIL import Image
folder_path = base_dir
extensions = []
corupt_img_paths=[]
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        try:
            im = Image.open(file_path)
        except:
            print(file_path)
            corupt_img_paths.append(file_path)
            continue
        else:
            rgb_im = im.convert('RGB')
            if filee.split('.')[1] not in extensions:
                extensions.append(filee.split('.')[1])

/content/LEAF_BETA/validation_set/Healthy Leaves
/content/LEAF_BETA/validation_set/Stage1
/content/LEAF_BETA/validation_set/Stage4
/content/LEAF_BETA/validation_set/Stage2
/content/LEAF_BETA/validation_set/Stage3
/content/LEAF_BETA/training_set/Healthy Leaves
/content/LEAF_BETA/training_set/Stage1
/content/LEAF_BETA/training_set/Stage4
/content/LEAF_BETA/training_set/Stage2
/content/LEAF_BETA/training_set/Stage3


In [ ]:
m.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [ ]:
m.compile(tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = m.fit(train_generator, validation_data= validation_generator, epochs=100)

Epoch 1/100
63/63 [==============================] - 404s 6s/step - loss: 0.7326 - accuracy: 0.6515 - val_loss: 1.9787 - val_accuracy: 0.2860
Epoch 2/100
63/63 [==============================] - 395s 6s/step - loss: 0.6214 - accuracy: 0.6775 - val_loss: 2.0562 - val_accuracy: 0.3040
Epoch 3/100
63/63 [==============================] - 396s 6s/step - loss: 0.5667 - accuracy: 0.6865 - val_loss: 2.0546 - val_accuracy: 0.3380
Epoch 4/100
63/63 [==============================] - 417s 7s/step - loss: 0.4967 - accuracy: 0.7095 - val_loss: 1.9149 - val_accuracy: 0.3540
Epoch 5/100
63/63 [==============================] - 461s 7s/step - loss: 0.5013 - accuracy: 0.7165 - val_loss: 1.8802 - val_accuracy: 0.3320
Epoch 6/100
63/63 [==============================] - 408s 6s/step - loss: 0.4546 - accuracy: 0.7485 - val_loss: 1.8958 - val_accuracy: 0.3500
Epoch 7/100
63/63 [==============================] - 399s 6s/step - loss: 0.4071 - accuracy: 0.7835 - val_loss: 1.8920 - val_accuracy: 0.3480
Epoch 